In [1]:
import os
import sys
import math
import numpy as np
import pandas as pd
import pickle as pkl
import tifffile as tif
from keras.layers import Dense
from keras.layers import Conv2D
from multiprocessing import Pool
from keras.utils import Sequence
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.layers import Concatenate
from keras.layers import Merge
from collections import Counter
from keras.models import Sequential
from keras.preprocessing.text import one_hot
os.chdir("../../")

Using TensorFlow backend.


In [2]:
class ImageDataGenerator(Sequence):
    
    def __init__(self, x_metadata, y_metadata, batch_size, crop_size):
        self.x = x_metadata
        self.y = y_metadata
        self.batch_size = batch_size
        self.cp = crop_size
    
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return [np.array([np.transpose(np.array(tif.imread(file_name), dtype=int)/255.0,
                                     (1,2,0))[self.cp:-self.cp,self.cp:-self.cp,:]
                        for file_name in batch_x]), np.array([np.transpose(np.array(tif.imread(file_name), dtype=int)/255.0,
                                     (1,2,0))[self.cp:-self.cp,self.cp:-self.cp,:]
                        for file_name in batch_x])], np.array(batch_y)         

class CNN_Model:
    
    def __init__(self, directory):
        
        self.path = directory
        self.classes = sorted(list(os.listdir(self.path+"/train")))
        
        self.train_metadata_x = []
        self.train_metadata_y = []
        self.test_metadata_x = []
        self.test_metadata_y = []
        
        for cls in self.classes:
            for im in os.listdir(self.path+"/train/"+cls):
                self.train_metadata_x.append(self.path+"/train/"+cls+"/"+im)
        
        for cls in self.classes:
            for im in os.listdir(self.path+"/test/"+cls):
                self.test_metadata_x.append(self.path+"/test/"+cls+"/"+im)
        
        np.random.shuffle(self.train_metadata_x)
        np.random.shuffle(self.test_metadata_x)
        
        for p in self.train_metadata_x:
            y = np.zeros(10, dtype=int)
            y[self.classes.index(p.split("/")[3])] = 1
            self.train_metadata_y.append(y)
            
        for p in self.test_metadata_x:
            y = np.zeros(10, dtype=int)
            y[self.classes.index(p.split("/")[3])] = 1
            self.test_metadata_y.append(y)
    
    def model_create(self):
        '''
        # Branch 1
        branch1 = Sequential()
        # Step 1 - Convolution
        branch1.add(Conv2D(filters=32, kernel_size=(2, 2), input_shape = (32,32,33), activation = 'relu'))
        branch1.add(Conv2D(filters=96, kernel_size=(3, 3), activation = 'relu'))
        branch1.add(Conv2D(filters=256, kernel_size=(4, 4), activation = 'relu'))
        branch1.add(Conv2D(filters=576, kernel_size=(5, 5), activation = 'relu'))
        branch1.add(MaxPool2D(pool_size = (2, 2)))
        branch1.summary()
        
        # Branch 2
        branch2 = Sequential()
        # Step 1 - Convolution
        branch2.add(Conv2D(filters=96, kernel_size=(3, 3), input_shape = (32,32,33), activation = 'relu'))
        branch2.add(Conv2D(filters=288, kernel_size=(4, 4), activation = 'relu'))
        branch2.add(Conv2D(filters=576, kernel_size=(5, 5), activation = 'relu'))
        #branch2.add(Conv2D(filters=, kernel_size=(5, 5), activation = 'relu'))
        branch2.add(MaxPool2D(pool_size = (2, 2)))
        branch2.summary()
        
        # Step 3 - Flattening
        classifier = Sequential()
        classifier.add(Merge([branch1, branch2], mode='concat'))
        classifier.add(Flatten())
        # Step 4 - Full connection
        classifier.add(Dense(128, activation = 'relu'))
        classifier.add(Dense(128, activation = 'tanh'))
        classifier.add(Dense(10, activation = 'sigmoid'))
        # Compiling the CNN
        classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        classifier.summary()
        return classifier
        '''
        
        
        
    def fit_generator(self, num_epochs=10, batch_size=32, crop_size=16):        
        
        classifier = self.model_create()
        train_data = ImageDataGenerator(self.train_metadata_x, self.train_metadata_y, batch_size, crop_size)
        test_data = ImageDataGenerator(self.test_metadata_x, self.test_metadata_y, batch_size, crop_size)
        history = classifier.fit_generator(train_data, epochs=num_epochs, use_multiprocessing=True,shuffle=True)
        classifier.evaluate_generator(test_data, use_multiprocessing=True)

In [2]:
df = pd.read_csv("occurrences_train.csv")
#df = df[["class","order"]]
#encoded_class = [ one_hot(d,10) for d in list(df['class']) ]
#encoded_order = one_hot(df,)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (36,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def encoder(d):
    """
    Pass the input as a pandas series.
    """
    dic = {}
    names = sorted(list(d.unique()))
    for i in names:
        dic[i] = names.index(i)
    #d.replace(dic,inplace=True)
    return dic

In [31]:
e_class = encoder(df['class'])
e_order = encoder(df['order'])

In [7]:
type(encoder(df['class']))

dict

In [4]:
pkl.dump(encoder(df['class']),open("Data/class_encoding.pkl","wb"))
pkl.dump(encoder(df['order']),open("Data/order_encoding.pkl","wb"))
pkl.dump(encoder(df['family']),open("Data/family_encoding.pkl","wb"))
pkl.dump(encoder(df['genus']),open("Data/genus_encoding.pkl","wb"))
pkl.dump(df['species_glc_id'].unique(),open("Data/specie_encoding.pkl","wb"))

In [ ]:
ob = CNN_Model(directory="Data/Class wise Data")

In [ ]:
ob.fit_generator(num_epochs=15, batch_size=100, crop_size=16)

In [9]:
import operator

d= {1:1, 2:4, 5:10}

sorted(d.items(), key=operator.itemgetter(0))

[(1, 1), (2, 4), (5, 10)]